In [1]:
# install openai 1.
#!pip install openai==1.14.3

In [2]:
import openai
import json
import ast
import os

In [4]:
OPENAI_API_KEY="" #Please add
OPENAI_API_VERSION = "2024-02-15-preview"
OPENAI_API_ENDPOINT="" #Please add

## create an assistant

In [5]:
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version=OPENAI_API_VERSION,
    azure_endpoint = OPENAI_API_ENDPOINT
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-35-turbo" #You must replace this value with the deployment name for your model.
)

In [6]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_w7IKcuck8U145iVW3U4b6qUx",
  "created_at": 1711749835,
  "description": null,
  "file_ids": [],
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-35-turbo",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


## create a thread

In [9]:

thread = client.beta.threads.create()
print(thread)

Thread(id='thread_ZhpH3VG9Zvvt4VIqIftWAzLu', created_at=1711749893, metadata={}, object='thread')


In [11]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [12]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [14]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_4dTMWXFwShpELj3wNOMJp5kS",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1711749911,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_ZhpH3VG9Zvvt4VIqIftWAzLu"
    },
    {
      "id": "msg_yljrQOxBg5RELK0OVYCrewlQ",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1711749895,
      "file_ids": [],
      "incomplete_at": null

In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [16]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [17]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 0 seconds
{
  "data": [
    {
      "id": "msg_66cSs8GMtGy1VpwRmOSwJXNQ",
      "assistant_id": "asst_w7IKcuck8U145iVW3U4b6qUx",
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-TYCFXVcm27aHGlYIn7qd7dLt"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave. The x-axis represents the value of x and the y-axis represents the value of sin(x). The wave oscillates between -1 and 1."
          },
          "type": "text"
        }
      ],
      "created_at": 1711749958,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_mn4mW7kpS7W6oYO9HRPMLUaA",
      "status": null,
      "thread_id": "thread_ZhpH3VG9Zvvt4VIqIftWAzLu"
   